In [2]:
!pip install ace

In [3]:
import pandas as pd

In [4]:
df1945 = pd.read_excel('/content/geocoded_data_1945.xlsx')
df1975 = pd.read_excel('/content/geocoded_data_1975.xlsx')
df1985 = pd.read_excel('/content/geocoded_data_1985.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '/content/geocoded_data_1945.xlsx'

In [ ]:
# Renaming the columns to be more general
df1945.columns = [col.replace('_1945', '') for col in df1945.columns]
df1975.columns = [col.replace('_1975', '') for col in df1975.columns]
df1985.columns = [col.replace('_1985', '') for col in df1985.columns]

# Merging unique addresses into a single dataframe
unique_addresses = pd.DataFrame(pd.concat([
    df1945['Google_Standard_address'],
    df1975['Google_Standard_address'],
    df1985['Google_Standard_address']
]).unique(), columns=['Address'])

# Initializing new columns
unique_addresses['ID_1945'] = unique_addresses['Address'].apply(lambda x: df1945[df1945['Google_Standard_address'] == x]['ID'].tolist())
unique_addresses['times_repeated_1945'] = unique_addresses['ID_1945'].apply(len)
unique_addresses['ID_1975'] = unique_addresses['Address'].apply(lambda x: df1975[df1975['Google_Standard_address'] == x]['ID'].tolist())
unique_addresses['times_repeated_1975'] = unique_addresses['ID_1975'].apply(len)
unique_addresses['ID_1985'] = unique_addresses['Address'].apply(lambda x: df1985[df1985['Google_Standard_address'] == x]['ID'].tolist())
unique_addresses['times_repeated_1985'] = unique_addresses['ID_1985'].apply(len)

# Adding Description column
def generate_description(row):
    years = []
    if row['times_repeated_1945'] > 0:
        years.append('1945')
    if row['times_repeated_1975'] > 0:
        years.append('1975')
    if row['times_repeated_1985'] > 0:
        years.append('1985')

    if len(years) == 1:
        return f'only in {years[0]}'
    elif len(years) == 2:
        return f'only in {years[0]} and {years[1]}'
    elif len(years) == 3:
        return 'found in all'
    else:
        return 'not found'

unique_addresses['Description'] = unique_addresses.apply(generate_description, axis=1)

unique_addresses

unique_addresses.to_excel('Commons.xlsx')